In cell 3 change the directory to the one you extracted the results in.

## 1. Setup and Imports

In [ ]:
# Install required packages
%pip install geopandas shapely rtree requests fiona pyproj folium

In [ ]:
import geopandas as gpd
import pandas as pd
import json
import requests
import math
from pathlib import Path
from shapely.geometry import LineString, box, Point, Polygon, MultiPolygon
from shapely.ops import unary_union
import warnings
warnings.filterwarnings('ignore')

print("✓ Imports successful")

## 2. Configuration

In [ ]:
# Configuration
BUFFER_DISTANCE = 3  # meters - buffer for network segment matching within polygons
ZOOM_LEVEL = 19  # from structure.json

# Path to the data directory containing network/, polygons/, tiles/, and structure.json
DATA_DIR = Path('./example')  # Update this path to your data directory

print(f"Data directory: {DATA_DIR}")
print(f"Exists: {DATA_DIR.exists()}")

## 3. Load Data

In [ ]:
# Verify data directory exists
if not DATA_DIR.exists():
    raise FileNotFoundError(f"Data directory not found: {DATA_DIR}")

extract_dir = DATA_DIR

print(f"✓ Using data directory: {extract_dir}")
print("\nContents:")
for item in sorted(extract_dir.rglob('*')):
    if item.is_file() and (item.suffix in ['.json', '.csv', '.shp', '.dbf', '.shx', '.prj']):
        print(f"  {item.relative_to(extract_dir)}")

In [ ]:
# Load structure.json
structure_file = list(extract_dir.rglob('structure.json'))[0]
with open(structure_file, 'r') as f:
    structure = json.load(f)

print("Structure configuration:")
print(json.dumps(structure, indent=2))

In [ ]:
# Find and load tile info
tile_info_files = list(extract_dir.rglob('*_info.json'))
if not tile_info_files:
    # Load from CSV
    csv_files = list(extract_dir.rglob('*_info.csv'))
    if not csv_files:
        raise FileNotFoundError("No tile info file found (neither JSON nor CSV)")
    
    tiles_df = pd.read_csv(csv_files[0])
    
    # Convert CSV to dict format using correct column names
    tile_info = {
        'center': [
            (tiles_df['topleft_y'].min() + tiles_df['bottomright_y'].max()) / 2,  # lat
            (tiles_df['topleft_x'].min() + tiles_df['bottomright_x'].max()) / 2   # lon
        ],
        'zoom': ZOOM_LEVEL,
        'bbox': [
            tiles_df['bottomright_y'].min(),  # min_lat
            tiles_df['bottomright_x'].min(),  # min_lon
            tiles_df['topleft_y'].max(),      # max_lat
            tiles_df['topleft_x'].max()       # max_lon
        ]
    }
else:
    # Load from JSON
    with open(tile_info_files[0], 'r') as f:
        tile_info_json = json.load(f)
    
    # Calculate center from bbox
    bbox = tile_info_json['bbox']
    tile_info = {
        'center': [
            (bbox[0] + bbox[1]) / 2,  # lat (average of min and max lat)
            (bbox[2] + bbox[3]) / 2   # lon (average of min and max lon)
        ],
        'zoom': tile_info_json.get('zoom', ZOOM_LEVEL),
        'bbox': [
            bbox[0],  # min_lat
            bbox[2],  # min_lon
            bbox[1],  # max_lat
            bbox[3]   # max_lon
        ]
    }
    
    # Load CSV for per-tile analysis
    csv_files = list(extract_dir.rglob('*_info.csv'))
    if csv_files:
        tiles_df = pd.read_csv(csv_files[0])
    else:
        tiles_df = None

print("\nTile Info:")
print(f"  Center: {tile_info['center']}")
print(f"  Zoom: {tile_info['zoom']}")
print(f"  BBox: {tile_info['bbox']}")

if tiles_df is not None:
    print(f"\n✓ Loaded {len(tiles_df)} tiles for per-tile analysis")

In [ ]:
# Load ML-detected network (paths)
network_shp = list(extract_dir.rglob('*Network*.shp'))[0]
ml_network = gpd.read_file(network_shp)

# Ensure CRS is WGS84
if ml_network.crs is None:
    ml_network = ml_network.set_crs('EPSG:4326')
elif ml_network.crs.to_string() != 'EPSG:4326':
    ml_network = ml_network.to_crs('EPSG:4326')

print(f"\n✓ Loaded ML Network: {len(ml_network)} features")
print(f"  Geometry types: {ml_network.geometry.type.value_counts().to_dict()}")
print(f"  CRS: {ml_network.crs}")
print(f"  Bounds: {ml_network.total_bounds}")

In [ ]:
# Load ML-detected polygons
polygon_shp = list(extract_dir.rglob('*Polygon*.shp'))[0]
ml_polygons = gpd.read_file(polygon_shp)

if ml_polygons.crs is None:
    ml_polygons = ml_polygons.set_crs('EPSG:4326')
elif ml_polygons.crs.to_string() != 'EPSG:4326':
    ml_polygons = ml_polygons.to_crs('EPSG:4326')

print(f"\n✓ Loaded ML Polygons: {len(ml_polygons)} features")
print(f"  Geometry types: {ml_polygons.geometry.type.value_counts().to_dict()}")
print(f"  CRS: {ml_polygons.crs}")
print(f"  Bounds: {ml_polygons.total_bounds}")

## 4. Fetch OSM Ground Truth Data

In [ ]:
def fetch_osm_paths(bbox):
    """
    Fetch OSM pedestrian paths using Overpass API
    bbox: [min_lat, min_lon, max_lat, max_lon]
    """
    overpass_query = f"""
    [out:json][timeout:60];
    (
      way["highway"="footway"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      way["highway"="pedestrian"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      way["highway"="path"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      way["highway"="steps"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      way["footway"="sidewalk"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      way["sidewalk"="both"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      way["sidewalk"="left"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      way["sidewalk"="right"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      way["sidewalk"="yes"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
    );
    out geom;
    """
    
    overpass_url = "https://overpass-api.de/api/interpreter"
    
    print(f"Querying Overpass API for bbox: {bbox}")
    print("This may take 10-30 seconds...")
    
    try:
        response = requests.post(overpass_url, data={'data': overpass_query}, timeout=90)
        response.raise_for_status()
        data = response.json()
        
        print(f"✓ Received {len(data.get('elements', []))} OSM elements")
        return data
    except Exception as e:
        print(f"✗ Error fetching OSM data: {e}")
        return None

# Fetch OSM data
osm_data = fetch_osm_paths(tile_info['bbox'])

In [ ]:
def osm_to_geodataframe(osm_data, bbox=None, segment_length=30):
    """
    Convert Overpass API JSON to GeoDataFrame
    Clips paths to bbox and splits long ways into segments
    
    Args:
        osm_data: Raw Overpass API response
        bbox: Bounding box [min_lat, min_lon, max_lat, max_lon] to clip to
        segment_length: Maximum segment length in meters (default 30m)
    """
    features = []
    
    # Create a polygon from bbox for clipping
    clip_box = None
    if bbox:
        clip_box = box(bbox[1], bbox[0], bbox[3], bbox[2])  # (minlon, minlat, maxlon, maxlat)
    
    for element in osm_data.get('elements', []):
        if element['type'] == 'way' and 'geometry' in element:
            coords = [(node['lon'], node['lat']) for node in element['geometry']]
            
            if len(coords) >= 2:
                tags = element.get('tags', {})
                osm_id = element['id']

                # Check if this is a cyclic area (closed polygon)
                is_closed = coords[0] == coords[-1]
                is_area = tags.get('area') == 'yes' and is_closed
                # or ( tags.get('highway') == 'pedestrian' and len(coords) > 4 and is_closed )
                # but doesn't seem to actually fit OSM standards

                if is_area:
                    # Skip polygonal pedestrian areas - only validate linear paths
                    continue
                
                # Create LineString for this way
                way_line = LineString(coords)
                
                # Clip to bbox if provided
                if clip_box:
                    try:
                        clipped = way_line.intersection(clip_box)
                        # Handle different geometry types from intersection
                        if clipped.is_empty:
                            continue
                        if clipped.geom_type == 'LineString':
                            way_line = clipped
                        elif clipped.geom_type == 'MultiLineString':
                            # For MultiLineString, process each part separately
                            for geom in clipped.geoms:
                                if len(geom.coords) >= 2:
                                    segments = split_linestring_by_length(geom, segment_length)
                                    for seg_idx, segment in enumerate(segments):
                                        features.append({
                                            'geometry': segment,
                                            'osm_id': osm_id,
                                            'segment_index': seg_idx,
                                            'highway': tags.get('highway', ''),
                                            'footway': tags.get('footway', ''),
                                            'sidewalk': tags.get('sidewalk', ''),
                                            'name': tags.get('name', '')
                                        })
                            continue
                        else:
                            continue
                    except Exception as e:
                        print(f"Warning: Failed to clip way {osm_id}: {e}")
                        continue
                
                # Split the way into segments
                segments = split_linestring_by_length(way_line, segment_length)
                
                # Add each segment as a separate feature
                for seg_idx, segment in enumerate(segments):
                    if len(segment.coords) >= 2:
                        features.append({
                            'geometry': segment,
                            'osm_id': osm_id,
                            'segment_index': seg_idx,
                            'highway': tags.get('highway', ''),
                            'footway': tags.get('footway', ''),
                            'sidewalk': tags.get('sidewalk', ''),
                            'name': tags.get('name', '')
                        })
    
    if not features:
        print("⚠️ No valid OSM paths found!")
        return gpd.GeoDataFrame()
    
    gdf = gpd.GeoDataFrame(features, crs='EPSG:4326')
    return gdf

def split_linestring_by_length(line, max_segment_length):
    """
    Split a LineString into segments of approximately max_segment_length meters
    
    Args:
        line: shapely LineString in WGS84
        max_segment_length: Maximum length per segment in meters
        
    Returns:
        List of LineString segments
    """
    # Project to UTM for accurate length measurements
    line_utm = gpd.GeoSeries([line], crs='EPSG:4326').to_crs('EPSG:32619')[0]
    
    total_length = line_utm.length
    
    # If line is shorter than max segment, return as-is
    if total_length <= max_segment_length:
        return [line]
    
    # Calculate number of segments needed
    num_segments = math.ceil(total_length / max_segment_length)
    segment_length = total_length / num_segments
    
    segments = []
    
    # Extract points along the line at regular intervals
    for i in range(num_segments):
        start_dist = i * segment_length
        end_dist = (i + 1) * segment_length
        
        # Extract segment endpoints using interpolate
        start_point = line_utm.interpolate(start_dist)
        end_dist_actual = min(end_dist, total_length)
        end_point = line_utm.interpolate(end_dist_actual)
        
        # Build coordinate list for this segment
        coords = [start_point.coords[0]]
        
        # Add all original coordinates that fall within this segment
        for coord in line_utm.coords:
            point = Point(coord)
            dist_along = line_utm.project(point)
            if start_dist < dist_along < end_dist_actual:
                coords.append(coord)
        
        # Add end point
        coords.append(end_point.coords[0])
        
        # Remove duplicates while preserving order
        unique_coords = []
        for coord in coords:
            if not unique_coords or coord != unique_coords[-1]:
                unique_coords.append(coord)
        
        # Create segment line
        if len(unique_coords) >= 2:
            segment_utm = LineString(unique_coords)
            # Convert back to WGS84
            segment_wgs84 = gpd.GeoSeries([segment_utm], crs='EPSG:32619').to_crs('EPSG:4326')[0]
            segments.append(segment_wgs84)
    
    return segments if segments else [line]

# Convert to GeoDataFrame with segmentation and clipping
if osm_data:
    osm_paths = osm_to_geodataframe(osm_data, bbox=tile_info['bbox'], segment_length=1)
    print(f"\n✓ Created OSM GeoDataFrame: {len(osm_paths)} features")
    print(f"  Original ways: {osm_paths['osm_id'].nunique()}")
    print(f"  Highway types: {osm_paths['highway'].value_counts().to_dict()}")
    print(f"  Bounds: {osm_paths.total_bounds}")
else:
    osm_paths = gpd.GeoDataFrame()

## 4.5. Area Composition Analysis Functions

In [ ]:
def fetch_osm_area_features(bbox):
    """
    Fetch OSM features for area composition analysis
    bbox: [min_lat, min_lon, max_lat, max_lon]
    """
    overpass_query = f"""
    [out:json][timeout:60];
    (
      way["building"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      relation["building"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      
      way["highway"~"motorway|trunk|primary|secondary|tertiary|residential|service|unclassified"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      
      way["highway"="pedestrian"]["area"="yes"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      relation["highway"="pedestrian"]["area"="yes"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      
      way["landuse"~"grass|meadow|recreation_ground"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      way["leisure"~"park|garden|playground"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      way["natural"~"grassland|scrub"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      relation["landuse"~"grass|meadow|recreation_ground"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      relation["leisure"~"park|garden|playground"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      
      way["natural"="water"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      way["waterway"~"river|stream"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      relation["natural"="water"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      
      way["railway"~"rail|tram"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      
      way["amenity"="parking"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      relation["amenity"="parking"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
    );
    out geom;
    """
    
    overpass_url = "https://overpass-api.de/api/interpreter"
    
    try:
        response = requests.post(overpass_url, data={'data': overpass_query}, timeout=90)
        response.raise_for_status()
        data = response.json()
        return data
    except Exception as e:
        print(f"⚠️ Error fetching OSM area features: {e}")
        return None

In [ ]:
def estimate_road_width(tags):
    """
    Estimate road width in meters based on OSM tags
    """
    # Check for explicit width tag
    if 'width' in tags:
        try:
            return float(tags['width'])
        except:
            pass
    
    highway_type = tags.get('highway', '')
    lanes = tags.get('lanes', None)
    
    # Lane width assumptions
    lane_widths = {
        'motorway': 3.5,
        'trunk': 3.5,
        'primary': 3.25,
        'secondary': 3.25,
        'tertiary': 3.0,
        'residential': 3.0,
        'unclassified': 3.0,
        'service': 2.5
    }
    
    # Default lane counts if not specified
    default_lanes = {
        'motorway': 2,
        'trunk': 2,
        'primary': 2,
        'secondary': 2,
        'tertiary': 1,
        'residential': 1,
        'unclassified': 1,
        'service': 1
    }
    
    lane_width = lane_widths.get(highway_type, 3.0)
    
    if lanes:
        try:
            num_lanes = int(lanes)
            return lane_width * num_lanes
        except:
            pass
    
    return lane_width * default_lanes.get(highway_type, 1)


def estimate_railway_width(tags):
    """
    Estimate railway width in meters based on OSM tags
    """
    tracks = tags.get('tracks', '1')
    try:
        num_tracks = int(tracks)
        return 1.5 * num_tracks  # Standard gauge track width
    except:
        return 1.5

In [ ]:
def compute_tile_area_composition(tile_bbox, osm_features_data):
    """
    Compute area composition for a tile
    
    Args:
        tile_bbox: shapely box for the tile
        osm_features_data: OSM Overpass API response with area features
    
    Returns:
        dict with area percentages for each category
    """
    if not osm_features_data or 'elements' not in osm_features_data:
        return None
    
    # Calculate total tile area (in square meters)
    tile_gdf = gpd.GeoDataFrame([{'geometry': tile_bbox}], crs='EPSG:4326')
    tile_utm = tile_gdf.to_crs('EPSG:32619')
    total_area = tile_utm.geometry.area.sum()
    
    # Initialize category geometries
    categories = {
        'buildings': [],
        'roads': [],
        'pedestrian_areas': [],
        'green_spaces': [],
        'water': [],
        'railways': [],
        'parking': []
    }
    
    for element in osm_features_data['elements']:
        tags = element.get('tags', {})
        geom = None
        
        # Parse geometry based on element type
        if element['type'] == 'way' and 'geometry' in element:
            coords = [(node['lon'], node['lat']) for node in element['geometry']]
            
            # Buildings
            if 'building' in tags:
                if len(coords) >= 3:
                    geom = Polygon(coords) if coords[0] == coords[-1] else Polygon(coords + [coords[0]])
                    categories['buildings'].append(geom)
            
            # Roads (convert to buffered polygon)
            elif 'highway' in tags and tags['highway'] in ['motorway', 'trunk', 'primary', 'secondary', 
                                                            'tertiary', 'residential', 'service', 'unclassified']:
                if len(coords) >= 2:
                    line = LineString(coords)
                    # Clip to tile
                    line = line.intersection(tile_bbox)
                    if not line.is_empty:
                        width = estimate_road_width(tags)
                        # Buffer in UTM for accurate width
                        line_gdf = gpd.GeoDataFrame([{'geometry': line}], crs='EPSG:4326')
                        line_utm = line_gdf.to_crs('EPSG:32619').geometry[0]
                        buffered = line_utm.buffer(width / 2)
                        # Convert back to WGS84
                        buffered_gdf = gpd.GeoDataFrame([{'geometry': buffered}], crs='EPSG:32619')
                        geom = buffered_gdf.to_crs('EPSG:4326').geometry[0]
                        categories['roads'].append(geom)
            
            # Pedestrian areas
            elif tags.get('highway') == 'pedestrian' and tags.get('area') == 'yes':
                if len(coords) >= 3:
                    geom = Polygon(coords) if coords[0] == coords[-1] else Polygon(coords + [coords[0]])
                    categories['pedestrian_areas'].append(geom)
            
            # Green spaces
            elif (tags.get('landuse') in ['grass', 'meadow', 'recreation_ground'] or
                  tags.get('leisure') in ['park', 'garden', 'playground'] or
                  tags.get('natural') in ['grassland', 'scrub']):
                if len(coords) >= 3:
                    geom = Polygon(coords) if coords[0] == coords[-1] else Polygon(coords + [coords[0]])
                    categories['green_spaces'].append(geom)
            
            # Water
            elif tags.get('natural') == 'water':
                if len(coords) >= 3:
                    geom = Polygon(coords) if coords[0] == coords[-1] else Polygon(coords + [coords[0]])
                    categories['water'].append(geom)
            elif tags.get('waterway') in ['river', 'stream']:
                if len(coords) >= 2:
                    line = LineString(coords)
                    line = line.intersection(tile_bbox)
                    if not line.is_empty:
                        # Buffer waterways by 5m default
                        line_gdf = gpd.GeoDataFrame([{'geometry': line}], crs='EPSG:4326')
                        line_utm = line_gdf.to_crs('EPSG:32619').geometry[0]
                        buffered = line_utm.buffer(5)
                        buffered_gdf = gpd.GeoDataFrame([{'geometry': buffered}], crs='EPSG:32619')
                        geom = buffered_gdf.to_crs('EPSG:4326').geometry[0]
                        categories['water'].append(geom)
            
            # Railways
            elif tags.get('railway') in ['rail', 'tram']:
                if len(coords) >= 2:
                    line = LineString(coords)
                    line = line.intersection(tile_bbox)
                    if not line.is_empty:
                        width = estimate_railway_width(tags)
                        line_gdf = gpd.GeoDataFrame([{'geometry': line}], crs='EPSG:4326')
                        line_utm = line_gdf.to_crs('EPSG:32619').geometry[0]
                        buffered = line_utm.buffer(width / 2)
                        buffered_gdf = gpd.GeoDataFrame([{'geometry': buffered}], crs='EPSG:32619')
                        geom = buffered_gdf.to_crs('EPSG:4326').geometry[0]
                        categories['railways'].append(geom)
            
            # Parking
            elif tags.get('amenity') == 'parking':
                if len(coords) >= 3:
                    geom = Polygon(coords) if coords[0] == coords[-1] else Polygon(coords + [coords[0]])
                    categories['parking'].append(geom)
    
    # Calculate areas
    result = {}
    covered_area = 0
    
    for category, geoms in categories.items():
        if geoms:
            # Clip to tile and calculate area
            clipped_geoms = []
            for geom in geoms:
                try:
                    clipped = geom.intersection(tile_bbox)
                    if not clipped.is_empty:
                        clipped_geoms.append(clipped)
                except:
                    pass
            
            if clipped_geoms:
                # Union and convert to UTM for accurate area
                category_gdf = gpd.GeoDataFrame([{'geometry': g} for g in clipped_geoms], crs='EPSG:4326')
                category_utm = category_gdf.to_crs('EPSG:32619')
                
                # Union all geometries in category to avoid double-counting overlaps
                try:
                    unified = unary_union(category_utm.geometry)
                    area = unified.area if hasattr(unified, 'area') else 0
                except:
                    area = category_utm.geometry.area.sum()
                
                covered_area += area
                result[f'{category}_area_sqm'] = round(area, 2)
                result[f'{category}_pct'] = round((area / total_area) * 100, 2) if total_area > 0 else 0
            else:
                result[f'{category}_area_sqm'] = 0
                result[f'{category}_pct'] = 0
        else:
            result[f'{category}_area_sqm'] = 0
            result[f'{category}_pct'] = 0
    
    # Calculate unmapped percentage
    unmapped_area = max(0, total_area - covered_area)
    result['unmapped_area_sqm'] = round(unmapped_area, 2)
    result['unmapped_pct'] = round((unmapped_area / total_area) * 100, 2) if total_area > 0 else 0
    result['total_area_sqm'] = round(total_area, 2)
    
    return result

## 5. Polygon-Based Confusion Matrix Computation

In [ ]:
def compute_length_meters(gdf):
    """
    Compute total length in meters using a projected CRS
    """
    if len(gdf) == 0:
        return 0.0
    
    # Project to UTM Zone 19N for accurate length calculation
    gdf_projected = gdf.to_crs('EPSG:32619')
    return gdf_projected.geometry.length.sum()

# Calculate initial lengths
ml_length = compute_length_meters(ml_network)
osm_length = compute_length_meters(osm_paths)
ml_polygon_area = ml_polygons.to_crs('EPSG:32619').geometry.area.sum()

print(f"ML Network Total Length: {ml_length:.2f} meters")
print(f"OSM Paths Total Length: {osm_length:.2f} meters")
print(f"ML Polygons Total Area: {ml_polygon_area:.2f} square meters")
print(f"ML Polygons Count: {len(ml_polygons)}")

In [ ]:
def compute_polygon_based_confusion_matrix(ml_network_gdf, ml_polygons_gdf, osm_gdf, buffer_dist=3):
    """
    Compute confusion matrix using polygon-based validation
    
    Logic:
    1. True Positive (TP): ML network segments within ML polygons that also contain OSM segments
    2. False Negative (FN): OSM segments outside all ML polygons
    3. False Positive (FP): ML network segments within ML polygons that contain no OSM segments

    OSM segments are split by polygon boundaries so that parts inside and outside polygons
    are evaluated separately. This handles cases where a path spans multiple polygons.
    
    Args:
        ml_network_gdf: ML detected network paths GeoDataFrame
        ml_polygons_gdf: ML detected polygons GeoDataFrame
        osm_gdf: OSM ground truth paths GeoDataFrame
        buffer_dist: Buffer distance in meters for segment matching within polygons
    
    Returns:
        tp_gdf, fp_gdf, fn_gdf, tp_osm_gdf, metrics
    """
    print(f"\nComputing polygon-based confusion matrix (buffer={buffer_dist}m)...")
    
    if len(ml_network_gdf) == 0 or len(osm_gdf) == 0 or len(ml_polygons_gdf) == 0:
        print("⚠️ One or more datasets are empty!")
        return None, None, None, None
    
    # Project all data to UTM for accurate spatial operations
    print("  Projecting to UTM...")
    ml_network_utm = ml_network_gdf.to_crs('EPSG:32619')
    ml_polygons_utm = ml_polygons_gdf.to_crs('EPSG:32619')
    osm_utm = osm_gdf.to_crs('EPSG:32619')
    
    # Create unified polygon coverage (union of all ML polygons)
    print("  Creating unified polygon coverage...")
    unified_polygon = unary_union(ml_polygons_utm.geometry)
    
    # Step 1: Split OSM segments by polygon boundaries
    print("  Splitting OSM segments by polygon boundaries...")
    osm_segments_inside = []
    osm_segments_outside = []
    
    for idx, osm_geom in enumerate(osm_utm.geometry):
        try:
            # Get the part inside polygons
            inside_part = osm_geom.intersection(unified_polygon)
            
            # Get the part outside polygons
            outside_part = osm_geom.difference(unified_polygon)
            
            # Add inside parts (can be LineString or MultiLineString)
            if not inside_part.is_empty:
                if inside_part.geom_type == 'LineString':
                    if len(inside_part.coords) >= 2:
                        osm_segments_inside.append({
                            'geometry': inside_part,
                            'original_idx': idx
                        })
                elif inside_part.geom_type == 'MultiLineString':
                    for line in inside_part.geoms:
                        if len(line.coords) >= 2:
                            osm_segments_inside.append({
                                'geometry': line,
                                'original_idx': idx
                            })
            
            # Add outside parts
            if not outside_part.is_empty:
                if outside_part.geom_type == 'LineString':
                    if len(outside_part.coords) >= 2:
                        osm_segments_outside.append({
                            'geometry': outside_part,
                            'original_idx': idx
                        })
                elif outside_part.geom_type == 'MultiLineString':
                    for line in outside_part.geoms:
                        if len(line.coords) >= 2:
                            osm_segments_outside.append({
                                'geometry': line,
                                'original_idx': idx
                            })
        except Exception as e:
            print(f"    Warning: Failed to split OSM segment {idx}: {e}")
            # If splitting fails, classify the whole segment based on centroid
            if osm_geom.centroid.within(unified_polygon):
                osm_segments_inside.append({
                    'geometry': osm_geom,
                    'original_idx': idx
                })
            else:
                osm_segments_outside.append({
                    'geometry': osm_geom,
                    'original_idx': idx
                })
    
    print(f"    Split into {len(osm_segments_inside)} inside parts and {len(osm_segments_outside)} outside parts")
    
    # Create GeoDataFrames for inside and outside OSM segments
    if osm_segments_inside:
        osm_inside_gdf = gpd.GeoDataFrame(
            osm_segments_inside, 
            crs='EPSG:32619'
        )
    else:
        osm_inside_gdf = gpd.GeoDataFrame(columns=['geometry', 'original_idx'], crs='EPSG:32619')
    
    if osm_segments_outside:
        osm_outside_gdf = gpd.GeoDataFrame(
            osm_segments_outside,
            crs='EPSG:32619'
        )
        # Convert back to WGS84 for output
        fn_gdf = osm_outside_gdf.to_crs('EPSG:4326')
        fn_length = osm_outside_gdf.geometry.length.sum()
    else:
        fn_gdf = gpd.GeoDataFrame(columns=['geometry', 'original_idx'], crs='EPSG:4326')
        fn_length = 0.0
    
    print(f"    False Negative length (outside polygons): {fn_length:.2f}m")
    
    # Step 2: Find ML network segments within polygons
    print("  Filtering ML network segments within polygons...")
    ml_in_polygon_indices = []
    for idx, ml_geom in enumerate(ml_network_utm.geometry):
        if ml_geom.intersects(unified_polygon):
            ml_in_polygon_indices.append(idx)
    
    ml_in_polygons = ml_network_gdf.iloc[ml_in_polygon_indices].copy()
    ml_in_polygons_utm = ml_network_utm.iloc[ml_in_polygon_indices].copy()
    
    print(f"    Found {len(ml_in_polygons)} ML segments within polygons")
    
    # Step 3: Match ML segments with OSM segments inside polygons
    print("  Matching ML segments with OSM segments (with buffer)...")
    
    if len(osm_inside_gdf) == 0:
        # No OSM segments inside polygons, all ML segments are false positives
        tp_gdf = gpd.GeoDataFrame(columns=ml_network_gdf.columns, crs='EPSG:4326')
        fp_gdf = ml_in_polygons.copy()
        tp_osm_gdf = gpd.GeoDataFrame(columns=['geometry', 'original_idx'], crs='EPSG:4326')
        tp_length = 0.0
        fp_length = ml_in_polygons_utm.geometry.length.sum()
        total_osm_in_polygons = 0.0
    else:
        # Create buffered OSM segments for matching
        osm_buffered = osm_inside_gdf.copy()
        osm_buffered['geometry'] = osm_inside_gdf.geometry.buffer(buffer_dist)
        
        # Build spatial index
        osm_sindex = osm_buffered.sindex
        
        tp_indices = []
        fp_indices = []
        tp_osm_indices = set()  # Track which OSM segments were matched
        
        for idx, ml_geom in enumerate(ml_in_polygons_utm.geometry):
            # Find candidate OSM matches
            possible_matches = list(osm_sindex.intersection(ml_geom.bounds))
            
            has_match = False
            for osm_idx in possible_matches:
                osm_geom_buffered = osm_buffered.iloc[osm_idx].geometry
                
                # Check if ML segment intersects buffered OSM segment
                if ml_geom.intersects(osm_geom_buffered):
                    has_match = True
                    tp_osm_indices.add(osm_idx)  # Track matched OSM segment
                    break
            
            if has_match:
                tp_indices.append(ml_in_polygon_indices[idx])
            else:
                fp_indices.append(ml_in_polygon_indices[idx])
        
        # Create result GeoDataFrames
        tp_gdf = ml_network_gdf.iloc[tp_indices].copy()
        fp_gdf = ml_network_gdf.iloc[fp_indices].copy()
        # Create OSM TP GeoDataFrame from matched segments
        if tp_osm_indices:
            tp_osm_gdf = osm_inside_gdf.iloc[list(tp_osm_indices)].copy().to_crs('EPSG:4326')
        else:
            tp_osm_gdf = gpd.GeoDataFrame(columns=['geometry', 'original_idx'], crs='EPSG:4326')
        
        # Calculate lengths
        tp_length = ml_network_utm.iloc[[ml_in_polygon_indices.index(i) for i in tp_indices]].geometry.length.sum() if tp_indices else 0.0
        fp_length = ml_network_utm.iloc[[ml_in_polygon_indices.index(i) for i in fp_indices]].geometry.length.sum() if fp_indices else 0.0
        total_osm_in_polygons = osm_inside_gdf.geometry.length.sum()
    
    # Calculate metrics
    total_ml_in_polygons = tp_length + fp_length
    
    precision = tp_length / total_ml_in_polygons if total_ml_in_polygons > 0 else 0
    recall = tp_length / (tp_length + fn_length) if (tp_length + fn_length) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    iou = tp_length / (tp_length + fp_length + fn_length) if (tp_length + fp_length + fn_length) > 0 else 0
    
    metrics = {
        'method': 'polygon_based_segmented',
        'buffer_distance_m': buffer_dist,
        'polygon_count': len(ml_polygons_gdf),
        'polygon_area_sqm': round(ml_polygon_area, 2),
        'tp_length_m': round(tp_length, 2),
        'fp_length_m': round(fp_length, 2),
        'fn_length_m': round(fn_length, 2),
        'tp_count': len(tp_gdf),
        'fp_count': len(fp_gdf),
        'fn_count': len(fn_gdf),
        'ml_total_length_m': round(compute_length_meters(ml_network_gdf), 2),
        'ml_in_polygon_length_m': round(total_ml_in_polygons, 2),
        'osm_total_length_m': round(compute_length_meters(osm_gdf), 2),
        'osm_in_polygon_length_m': round(total_osm_in_polygons, 2),
        'osm_outside_polygon_length_m': round(fn_length, 2),
        'osm_inside_parts_count': len(osm_segments_inside),
        'osm_outside_parts_count': len(osm_segments_outside),
        'precision': round(precision, 4),
        'recall': round(recall, 4),
        'f1_score': round(f1_score, 4),
        'iou': round(iou, 4)
    }

    print(f"\n✓ Polygon-based analysis complete (with segmentation):")
    print(f"  ML segments in polygons: {len(ml_in_polygons)} ({total_ml_in_polygons:.2f}m)")
    print(f"  OSM parts in polygons: {len(osm_segments_inside)} ({total_osm_in_polygons:.2f}m)")
    print(f"  OSM parts outside polygons: {len(osm_segments_outside)} ({fn_length:.2f}m)")
    print(f"\n  TP: {metrics['tp_count']} features, {metrics['tp_length_m']:.2f}m")
    print(f"  FP: {metrics['fp_count']} features, {metrics['fp_length_m']:.2f}m")
    print(f"  FN: {metrics['fn_count']} parts, {metrics['fn_length_m']:.2f}m")

    return tp_gdf, fp_gdf, fn_gdf, tp_osm_gdf, metrics

# Compute polygon-based confusion matrix
tp_global, fp_global, fn_global, tp_osm_global, metrics_global = compute_polygon_based_confusion_matrix(
    ml_network, ml_polygons, osm_paths, BUFFER_DISTANCE
)

## 6. Compute Per-Tile Statistics

In [ ]:
def compute_per_tile_metrics_polygon_based(ml_network_gdf, ml_polygons_gdf, osm_gdf, tiles_df, buffer_dist=3, include_area_composition=True):
    """
    Compute polygon-based confusion matrix for each tile with optional area composition analysis
    
    Args:
        include_area_composition: If True, fetch OSM features and compute area composition for each tile
    """
    if tiles_df is None or len(tiles_df) == 0:
        print("⚠️ No tile information available for per-tile analysis")
        return None
    
    print(f"\nComputing per-tile polygon-based metrics for {len(tiles_df)} tiles...")
    if include_area_composition:
        print("  (Including area composition analysis - this may take longer)")
    
    # Fetch OSM area features once for the entire study area
    osm_area_data = None
    if include_area_composition:
        print("  Fetching OSM area features for entire study area...")
        study_area_bbox = [
            tiles_df['bottomright_y'].min(),  # min_lat
            tiles_df['topleft_x'].min(),      # min_lon
            tiles_df['topleft_y'].max(),      # max_lat
            tiles_df['bottomright_x'].max()   # max_lon
        ]
        osm_area_data = fetch_osm_area_features(study_area_bbox)
        if osm_area_data:
            print(f"  ✓ Fetched {len(osm_area_data.get('elements', []))} OSM area features")
        else:
            print("  ⚠️ Failed to fetch OSM area features")
    
    tile_metrics = []
    
    for idx, tile in tiles_df.iterrows():
        if (idx + 1) % 10 == 0:
            print(f"  Processing tile {idx+1}/{len(tiles_df)}...")
        
        # Create bounding box for this tile
        tile_bbox = box(
            tile['topleft_x'],      # min_lon
            tile['bottomright_y'],  # min_lat
            tile['bottomright_x'],  # max_lon
            tile['topleft_y']       # max_lat
        )
        
        # Calculate tile center
        tile_center_lat = (tile['topleft_y'] + tile['bottomright_y']) / 2
        tile_center_lon = (tile['topleft_x'] + tile['bottomright_x']) / 2
        
        # Compute area composition if OSM data was fetched
        area_composition = None
        if include_area_composition and osm_area_data:
            area_composition = compute_tile_area_composition(tile_bbox, osm_area_data)
        
        # Filter features within this tile
        ml_network_tile = ml_network_gdf[ml_network_gdf.geometry.intersects(tile_bbox)].copy()
        ml_polygons_tile = ml_polygons_gdf[ml_polygons_gdf.geometry.intersects(tile_bbox)].copy()
        osm_tile = osm_gdf[osm_gdf.geometry.intersects(tile_bbox)].copy()
        
        if len(ml_network_tile) == 0 and len(osm_tile) == 0:
            # Empty tile
            tile_metric = {
                'tile_id': tile.get('idd', f"tile_{idx}"),
                'xtile': int(tile.get('xtile', -1)),
                'ytile': int(tile.get('ytile', -1)),
                'lat': tile_center_lat,
                'lon': tile_center_lon,
                'ml_network_count': 0,
                'ml_polygon_count': 0,
                'osm_count': 0,
                'tp_length_m': 0,
                'fp_length_m': 0,
                'fn_length_m': 0,
                'precision': 0,
                'recall': 0,
                'f1_score': 0,
                'iou': 0
            }
            # Add area composition if available
            if area_composition:
                tile_metric.update(area_composition)
            tile_metrics.append(tile_metric)
            continue
        
        # Compute confusion matrix for this tile
        if len(ml_network_tile) > 0 and len(osm_tile) > 0 and len(ml_polygons_tile) > 0:
            _, _, _, _, tile_metric = compute_polygon_based_confusion_matrix(
                ml_network_tile, ml_polygons_tile, osm_tile, buffer_dist
            )
            if tile_metric:
                tile_metric['tile_id'] = tile.get('idd', f"tile_{idx}")
                tile_metric['xtile'] = int(tile.get('xtile', -1))
                tile_metric['ytile'] = int(tile.get('ytile', -1))
                tile_metric['lat'] = tile_center_lat
                tile_metric['lon'] = tile_center_lon
                tile_metric['ml_network_count'] = len(ml_network_tile)
                tile_metric['ml_polygon_count'] = len(ml_polygons_tile)
                tile_metric['osm_count'] = len(osm_tile)
                # Add area composition if available
                if area_composition:
                    tile_metric.update(area_composition)
                tile_metrics.append(tile_metric)
        else:
            # Handle partial data cases
            ml_length = compute_length_meters(ml_network_tile) if len(ml_network_tile) > 0 else 0
            osm_length = compute_length_meters(osm_tile) if len(osm_tile) > 0 else 0
            
            tile_metric = {
                'tile_id': tile.get('idd', f"tile_{idx}"),
                'xtile': int(tile.get('xtile', -1)),
                'ytile': int(tile.get('ytile', -1)),
                'lat': tile_center_lat,
                'lon': tile_center_lon,
                'ml_network_count': len(ml_network_tile),
                'ml_polygon_count': len(ml_polygons_tile),
                'osm_count': len(osm_tile),
                'tp_length_m': 0,
                'fp_length_m': ml_length if len(ml_polygons_tile) > 0 else 0,
                'fn_length_m': osm_length,
                'precision': 0,
                'recall': 0,
                'f1_score': 0,
                'iou': 0
            }
            # Add area composition if available
            if area_composition:
                tile_metric.update(area_composition)
            tile_metrics.append(tile_metric)
    
    print(f"\n✓ Computed metrics for {len(tile_metrics)} tiles")
    return tile_metrics

# Compute per-tile metrics
per_tile_metrics = compute_per_tile_metrics_polygon_based(
    ml_network, ml_polygons, osm_paths, tiles_df, BUFFER_DISTANCE
)

## 6.5. Area Composition Analysis Summary

In [ ]:
# Display area composition statistics if available
if per_tile_metrics and 'buildings_pct' in per_tile_metrics[0]:
    print("="*70)
    print("AREA COMPOSITION ANALYSIS")
    print("="*70)
    
    # Calculate average percentages across all tiles
    categories = ['buildings', 'roads', 'pedestrian_areas', 'green_spaces', 'water', 'railways', 'parking', 'unmapped']
    
    print("\nAverage Area Composition Across All Tiles:")
    for cat in categories:
        pct_key = f'{cat}_pct'
        if pct_key in per_tile_metrics[0]:
            avg_pct = sum(t[pct_key] for t in per_tile_metrics) / len(per_tile_metrics)
            print(f"  {cat.replace('_', ' ').title():25s}: {avg_pct:6.2f}%")
    
    # Correlation analysis: area composition vs performance
    print("\n" + "="*70)
    print("CORRELATION: Area Composition vs Performance")
    print("="*70)
    
    # Filter tiles with valid F1 scores
    valid_tiles = [t for t in per_tile_metrics if t['f1_score'] > 0]
    
    if len(valid_tiles) > 10:
        import numpy as np
        
        print(f"\nAnalyzing {len(valid_tiles)} tiles with valid predictions...")
        
        for cat in categories:
            pct_key = f'{cat}_pct'
            if pct_key in valid_tiles[0]:
                pcts = [t[pct_key] for t in valid_tiles]
                f1_scores = [t['f1_score'] for t in valid_tiles]
                
                # Calculate Pearson correlation
                correlation = np.corrcoef(pcts, f1_scores)[0, 1]
                
                print(f"  {cat.replace('_', ' ').title():25s} vs F1: {correlation:+.3f}")
        
        # Find tiles with highest/lowest building coverage
        print("\n" + "-"*70)
        print("Extreme Cases:")
        print("-"*70)
        
        sorted_by_buildings = sorted(valid_tiles, key=lambda x: x['buildings_pct'], reverse=True)
        print(f"\nHighest Building Coverage:")
        for t in sorted_by_buildings[:3]:
            print(f"  Tile {t['tile_id']}: {t['buildings_pct']:.1f}% buildings, F1={t['f1_score']:.3f}")
        
        print(f"\nLowest Building Coverage:")
        for t in sorted_by_buildings[-3:]:
            print(f"  Tile {t['tile_id']}: {t['buildings_pct']:.1f}% buildings, F1={t['f1_score']:.3f}")
    else:
        print("\n⚠️ Not enough tiles with valid predictions for correlation analysis")
else:
    print("⚠️ Area composition data not available in per-tile metrics")

## 7. Export Results

In [ ]:
# Create output directory with polygon-specific name
output_dir = Path('./polygon_analysis_output')
output_dir.mkdir(exist_ok=True)

print("Exporting polygon-based results...")

# 1. Export global confusion matrix
if metrics_global:
    with open(output_dir / 'confusion_matrix_global_polygon_based.json', 'w') as f:
        json.dump(metrics_global, f, indent=2)
    print("  ✓ confusion_matrix_global_polygon_based.json")

# 2. Export per-tile metrics
if per_tile_metrics:
    with open(output_dir / 'confusion_matrix_per_tile_polygon_based.json', 'w') as f:
        json.dump(per_tile_metrics, f, indent=2)
    print("  ✓ confusion_matrix_per_tile_polygon_based.json")

# 3. Export True Positives (ML network)
if tp_global is not None and len(tp_global) > 0:
    tp_global['category'] = 'true_positive'
    tp_global.to_file(output_dir / 'true_positives_polygon_based.geojson', driver='GeoJSON')
    print(f"  ✓ true_positives_polygon_based.geojson ({len(tp_global)} features)")

# 3b. Export True Positives (OSM ground truth parts)
if tp_osm_global is not None and len(tp_osm_global) > 0:
    tp_osm_global['category'] = 'true_positive_osm'
    tp_osm_global.to_file(output_dir / 'true_positives_osm_polygon_based.geojson', driver='GeoJSON')
    print(f"  ✓ true_positives_osm_polygon_based.geojson ({len(tp_osm_global)} OSM parts)")

# 4. Export False Positives
if fp_global is not None and len(fp_global) > 0:
    fp_global['category'] = 'false_positive'
    fp_global.to_file(output_dir / 'false_positives_polygon_based.geojson', driver='GeoJSON')
    print(f"  ✓ false_positives_polygon_based.geojson ({len(fp_global)} features)")

# 5. Export False Negatives
if fn_global is not None and len(fn_global) > 0:
    fn_global['category'] = 'false_negative'
    fn_global.to_file(output_dir / 'false_negatives_polygon_based.geojson', driver='GeoJSON')
    print(f"  ✓ false_negatives_polygon_based.geojson ({len(fn_global)} features)")

# 6. Export OSM ground truth
if len(osm_paths) > 0:
    osm_paths.to_file(output_dir / 'osm_ground_truth.geojson', driver='GeoJSON')
    print(f"  ✓ osm_ground_truth.geojson ({len(osm_paths)} features)")

# 7. Export ML polygons for reference
if len(ml_polygons) > 0:
    ml_polygons.to_file(output_dir / 'ml_polygons.geojson', driver='GeoJSON')
    print(f"  ✓ ml_polygons.geojson ({len(ml_polygons)} features)")

print(f"\n✓ All files exported to: {output_dir}")

## 8. Summary Statistics

This analysis uses POLYGON-BASED validation WITH SEGMENTATION:

1. OSM ground truth paths are SPLIT by polygon boundaries into:
   - Parts INSIDE polygons (used for TP/FP evaluation)
   - Parts OUTSIDE polygons (counted as False Negatives)
2. True Positive (TP): ML network segments within ML polygons that 
   match OSM segment parts inside polygons (within buffer distance)
3. False Negative (FN): OSM segment PARTS that fall outside all ML polygons
   (missed detection areas)
4. False Positive (FP): ML network segments within ML polygons that 
   have NO matching OSM segments (over-detection within valid areas)

Key Improvements:
- Handles OSM paths spanning multiple polygons correctly
- Each OSM path is split by polygon boundaries for accurate evaluation
- A path fully covered by multiple polygons is considered fully detected
- Only the parts truly outside polygons count as false negatives

Area Composition Analysis:
- Fetches comprehensive OSM features per tile (buildings, roads, parks, etc.)
- Estimates road/railway widths from lane/track information
- Calculates percentage of tile area occupied by each feature category
- Correlates area composition with model performance metrics
- Helps explain performance variations (e.g., high building density limits 
  aerial imagery visibility, affecting ML predictions)

Benefits:
- Better handles intersection parsing issues
- More accurate recall calculation
- Polygons define "search areas" for validation
- Separates area detection (polygons) from path extraction (network)
- Identifies environmental factors affecting model performance

In [ ]:
# Display summary
print("="*70)
print("POLYGON-BASED CONFUSION MATRIX ANALYSIS - SUMMARY")
print("="*70)

if metrics_global:
    print("\nGLOBAL METRICS:")
    print(f"  Method: {metrics_global['method']}")
    print(f"  Buffer Distance: {metrics_global['buffer_distance_m']}m")
    print(f"  Polygon Count: {metrics_global['polygon_count']}")
    print(f"  Polygon Area: {metrics_global['polygon_area_sqm']:.2f} m²")
    print(f"\n  ML Network Total Length: {metrics_global['ml_total_length_m']}m")
    print(f"  ML Network in Polygons: {metrics_global['ml_in_polygon_length_m']}m")
    print(f"  OSM Network Total Length: {metrics_global['osm_total_length_m']}m")
    print(f"  OSM Network in Polygons: {metrics_global['osm_in_polygon_length_m']}m")
    print(f"  OSM Network Outside Polygons: {metrics_global['osm_outside_polygon_length_m']}m")
    print(f"\n  True Positives:  {metrics_global['tp_length_m']}m ({metrics_global['tp_count']} features)")
    print(f"  False Positives: {metrics_global['fp_length_m']}m ({metrics_global['fp_count']} features)")
    print(f"  False Negatives: {metrics_global['fn_length_m']}m ({metrics_global['fn_count']} features)")
    print(f"\n  Precision: {metrics_global['precision']*100:.2f}%")
    print(f"  Recall:    {metrics_global['recall']*100:.2f}%")
    print(f"  F1 Score:  {metrics_global['f1_score']*100:.2f}%")
    print(f"  IoU:       {metrics_global['iou']*100:.2f}%")

if per_tile_metrics:
    print(f"\nPER-TILE METRICS:")
    print(f"  Total Tiles: {len(per_tile_metrics)}")
    
    # Calculate average metrics
    avg_precision = sum(t['precision'] for t in per_tile_metrics) / len(per_tile_metrics)
    avg_recall = sum(t['recall'] for t in per_tile_metrics) / len(per_tile_metrics)
    avg_f1 = sum(t['f1_score'] for t in per_tile_metrics) / len(per_tile_metrics)
    avg_iou = sum(t['iou'] for t in per_tile_metrics) / len(per_tile_metrics)
    
    print(f"  Avg Precision: {avg_precision*100:.2f}%")
    print(f"  Avg Recall:    {avg_recall*100:.2f}%")
    print(f"  Avg F1 Score:  {avg_f1*100:.2f}%")
    print(f"  Avg IoU:       {avg_iou*100:.2f}%")
    
    # Find best and worst tiles
    sorted_by_f1 = sorted(per_tile_metrics, key=lambda x: x['f1_score'], reverse=True)
    if len(sorted_by_f1) > 0:
        best_tile = sorted_by_f1[0]
        worst_tile = sorted_by_f1[-1]
        print(f"\n  Best Tile: {best_tile['tile_id']} (F1={best_tile['f1_score']*100:.2f}%)")
        print(f"  Worst Tile: {worst_tile['tile_id']} (F1={worst_tile['f1_score']*100:.2f}%)")

## 9. Visualization Preview

In [ ]:
# Create interactive map with polygon-based results
import folium
from folium import GeoJson

# Create base map
center = tile_info['center']
m = folium.Map(location=center, zoom_start=17)

# Add ML polygons layer (as background context)
if len(ml_polygons) > 0:
    ml_poly_geojson = json.loads(ml_polygons.to_json())
    GeoJson(ml_poly_geojson, name='ML Polygons (Search Areas)', 
            style_function=lambda x: {
                'fillColor': 'lightblue',
                'color': 'blue',
                'weight': 1,
                'fillOpacity': 0.2
            }).add_to(m)

# Add ML network layer
if len(ml_network) > 0:
    ml_geojson = json.loads(ml_network.to_json())
    GeoJson(ml_geojson, name='ML Network', 
            style_function=lambda x: {'color': 'purple', 'weight': 2, 'opacity': 0.6}).add_to(m)

# Add OSM ground truth layer
if len(osm_paths) > 0:
    osm_geojson = json.loads(osm_paths.to_json())
    GeoJson(osm_geojson, name='OSM Ground Truth', 
            style_function=lambda x: {'color': 'gray', 'weight': 2, 'opacity': 0.5}).add_to(m)

# Add confusion matrix layers
if tp_global is not None and len(tp_global) > 0:
    tp_geojson = json.loads(tp_global.to_json())
    GeoJson(tp_geojson, name='True Positives (in polygons + matched)', 
            style_function=lambda x: {'color': 'green', 'weight': 3, 'opacity': 0.8}).add_to(m)

if fp_global is not None and len(fp_global) > 0:
    fp_geojson = json.loads(fp_global.to_json())
    GeoJson(fp_geojson, name='False Positives (in polygons + no match)', 
            style_function=lambda x: {'color': 'orange', 'weight': 3, 'opacity': 0.8}).add_to(m)

if fn_global is not None and len(fn_global) > 0:
    fn_geojson = json.loads(fn_global.to_json())
    GeoJson(fn_geojson, name='False Negatives (outside polygons)', 
            style_function=lambda x: {'color': 'red', 'weight': 3, 'opacity': 0.8}).add_to(m)

# Add bbox boundary
bbox_coords = [
    [tile_info['bbox'][0], tile_info['bbox'][1]],
    [tile_info['bbox'][0], tile_info['bbox'][3]],
    [tile_info['bbox'][2], tile_info['bbox'][3]],
    [tile_info['bbox'][2], tile_info['bbox'][1]],
    [tile_info['bbox'][0], tile_info['bbox'][1]]
]
folium.PolyLine(bbox_coords, color='black', weight=2, opacity=0.5, 
                popup='Study Area BBox').add_to(m)

folium.LayerControl().add_to(m)
m

## 10. Interactive Tile-Based Visualization

In [ ]:
# Create interactive tile map with performance metrics and area composition
if per_tile_metrics and tiles_df is not None:
    import folium
    from folium import GeoJson, Popup
    import branca.colormap as cm
    
    # Create base map centered on study area
    center = tile_info['center']
    tile_map = folium.Map(location=center, zoom_start=15)
    
    # Create color scale based on F1 scores
    f1_scores = [t['f1_score'] for t in per_tile_metrics if t['f1_score'] > 0]
    
    if f1_scores:
        min_f1 = min(f1_scores)
        max_f1 = max(f1_scores)
        
        # Create colormap from red (low) to green (high)
        colormap = cm.LinearColormap(
            colors=['#d73027', '#fee08b', '#1a9850'],
            vmin=min_f1,
            vmax=max_f1,
            caption='F1 Score'
        )
        
        # Add colormap to map
        tile_map.add_child(colormap)
        
        # Add tiles as rectangles with color based on F1 score
        for tile_metric in per_tile_metrics:
            # Get tile from tiles_df
            tile_row = tiles_df[tiles_df.get('idd', tiles_df.index) == tile_metric['tile_id']]
            if len(tile_row) == 0:
                # Try finding by index
                try:
                    tile_idx = int(tile_metric['tile_id'].replace('tile_', ''))
                    tile_row = tiles_df.iloc[[tile_idx]]
                except:
                    continue
            
            if len(tile_row) > 0:
                tile = tile_row.iloc[0]
                
                # Create tile boundary
                tile_coords = [
                    [tile['topleft_y'], tile['topleft_x']],
                    [tile['topleft_y'], tile['bottomright_x']],
                    [tile['bottomright_y'], tile['bottomright_x']],
                    [tile['bottomright_y'], tile['topleft_x']],
                    [tile['topleft_y'], tile['topleft_x']]
                ]
                
                # Determine color based on F1 score
                f1 = tile_metric['f1_score']
                if f1 > 0:
                    fill_color = colormap(f1)
                    opacity = 0.6
                else:
                    fill_color = '#cccccc'
                    opacity = 0.3
                
                # Create detailed popup with all metrics
                popup_html = f"""
                <div style="font-family: Arial; width: 350px;">
                    <h4 style="margin: 0 0 10px 0; color: #333;">Tile {tile_metric['tile_id']}</h4>
                    
                    <div style="background: #f0f0f0; padding: 8px; margin: 5px 0; border-radius: 4px;">
                        <b>Location:</b><br>
                        Position: ({tile_metric['xtile']}, {tile_metric['ytile']})<br>
                        Center: ({tile_metric['lat']:.6f}, {tile_metric['lon']:.6f})
                    </div>
                    
                    <div style="background: #e8f5e9; padding: 8px; margin: 5px 0; border-radius: 4px;">
                        <b>Performance Metrics:</b><br>
                        F1 Score: <b>{tile_metric['f1_score']:.3f}</b><br>
                        Precision: {tile_metric['precision']:.3f}<br>
                        Recall: {tile_metric['recall']:.3f}<br>
                        IoU: {tile_metric['iou']:.3f}
                    </div>
                    
                    <div style="background: #fff3e0; padding: 8px; margin: 5px 0; border-radius: 4px;">
                        <b>Detection Counts:</b><br>
                        ML Network: {tile_metric['ml_network_count']}<br>
                        ML Polygons: {tile_metric['ml_polygon_count']}<br>
                        OSM Paths: {tile_metric['osm_count']}
                    </div>
                    
                    <div style="background: #e1f5fe; padding: 8px; margin: 5px 0; border-radius: 4px;">
                        <b>Length Metrics (m):</b><br>
                        True Positives: {tile_metric['tp_length_m']:.1f}m<br>
                        False Positives: {tile_metric['fp_length_m']:.1f}m<br>
                        False Negatives: {tile_metric['fn_length_m']:.1f}m
                    </div>
                """
                
                # Add area composition if available
                if 'buildings_pct' in tile_metric:
                    popup_html += f"""
                    <div style="background: #f3e5f5; padding: 8px; margin: 5px 0; border-radius: 4px;">
                        <b>Area Composition:</b><br>
                        Buildings: {tile_metric['buildings_pct']:.1f}%<br>
                        Roads: {tile_metric['roads_pct']:.1f}%<br>
                        Pedestrian Areas: {tile_metric['pedestrian_areas_pct']:.1f}%<br>
                        Green Spaces: {tile_metric['green_spaces_pct']:.1f}%<br>
                        Water: {tile_metric['water_pct']:.1f}%<br>
                        Railways: {tile_metric['railways_pct']:.1f}%<br>
                        Parking: {tile_metric['parking_pct']:.1f}%<br>
                        Unmapped: {tile_metric['unmapped_pct']:.1f}%
                    </div>
                    """
                
                popup_html += "</div>"
                
                # Create polygon for tile
                folium.Polygon(
                    locations=tile_coords,
                    color='black',
                    weight=1,
                    fill=True,
                    fill_color=fill_color,
                    fill_opacity=opacity,
                    popup=folium.Popup(popup_html, max_width=400),
                    tooltip=f"Tile {tile_metric['tile_id']}: F1={f1:.3f}"
                ).add_to(tile_map)
        
        # Add study area boundary
        bbox_coords = [
            [tile_info['bbox'][0], tile_info['bbox'][1]],
            [tile_info['bbox'][0], tile_info['bbox'][3]],
            [tile_info['bbox'][2], tile_info['bbox'][3]],
            [tile_info['bbox'][2], tile_info['bbox'][1]],
            [tile_info['bbox'][0], tile_info['bbox'][1]]
        ]
        folium.PolyLine(
            bbox_coords, 
            color='black', 
            weight=3, 
            opacity=0.8,
            popup='Study Area Boundary'
        ).add_to(tile_map)
        
        print("✓ Created interactive tile visualization")
        print(f"  - {len(per_tile_metrics)} tiles displayed")
        print(f"  - Color scale: F1 {min_f1:.3f} (red) to {max_f1:.3f} (green)")
        print(f"  - Click on any tile to see detailed metrics\n")
        
        # Display the map
        display(tile_map)
    else:
        print("⚠️ No tiles with valid F1 scores to visualize")
else:
    print("⚠️ Per-tile metrics or tiles dataframe not available for visualization")